In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive

In [ ]:
!pip install ultralytics  # 安装官方库，支持YOLOv8的所有功能

In [ ]:
!unzip /content/drive/MyDrive/underwater_dataset.zip -d /content/underwater_dataset
# 解压后路径：/content/underwater_dataset/underwater_dataset/train/...

In [ ]:
# 写入配置文件
with open('/content/underwater.yaml', 'w') as f:
    f.write('''
train: /content/underwater_dataset/underwater_dataset/train/images  # 训练图像路径
val: /content/underwater_dataset/underwater_dataset/val/images      # 验证图像路径
nc: 2  # 类别数量（礁石、管道）
names: ['rock', 'pipe']  # 类别名称（与标注时一致）
    ''')

In [ ]:
from ultralytics import YOLO

# 加载YOLOv8轻量级预训练模型（nano版本，参数最少，适合新手）
model = YOLO('yolov8n.pt')

In [ ]:
# 训练模型：epochs=10（轮次），batch=8（批次大小），imgsz=640（图像尺寸）
results = model.train(
    data='/content/underwater.yaml',  # 数据集配置文件
    epochs=10,                        # 训练轮次（小数据集10-20轮足够）
    batch=8,                          # 每次输入8张图（适配T4 GPU）
    imgsz=640,                        # 图像缩放为640x640
    device=0,                         # 使用第0号GPU
    project='underwater_runs',        # 结果保存文件夹名称
    name='first_train'                # 本次训练名称
)

In [ ]:
# 选择一张验证集图像（可通过!ls /content/underwater_dataset/underwater_dataset/val/images查看）
test_image_path = '/content/underwater_dataset/underwater_dataset/val/images/img_96.jpg'

# 用训练好的模型预测
results = model(test_image_path)

# 保存带检测框的结果图
for r in results:
    r.save(filename='/content/test_result_85.jpg')  # 保存路径

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# 显示结果图
img = Image.open('/content/test_result_85.jpg')
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')  # 关闭坐标轴
plt.show()

In [ ]:
# 正确加载训练好的模型（替换为你的权重路径）
model = YOLO('/content/underwater_runs/first_train/weights/best.pt')

In [ ]:
!find /content -name "best.pt"  # 搜索Colab中所有best.pt文件

In [ ]:
print(f"训练集mAP@0.5: {results.box.map50:.3f}")  # 应>0.3，否则模型未学好

In [ ]:
import pandas as pd

# 日志文件路径（根据你的训练结果文件夹调整）
log_path = '/content/underwater_runs/first_train/results.csv'

# 读取日志
df = pd.read_csv(log_path)

# 打印最后一轮的mAP@0.5（最接近训练完成的指标）
print(f"最后一轮mAP@0.5: {df['metrics/mAP50(B)'].iloc[-1]:.3f}")

In [ ]:
# 增加训练轮次（如epochs=30），让模型充分学习
model.train(data='/content/underwater.yaml', epochs=50, batch=8, imgsz=640)

In [ ]:
import pandas as pd

# 日志文件路径（根据你的训练结果文件夹调整）
log_path = '/content/underwater_runs/first_train/results.csv'

# 读取日志
df = pd.read_csv(log_path)

# 打印最后一轮的mAP@0.5（最接近训练完成的指标）
print(f"最后一轮mAP@0.5: {df['metrics/mAP50(B)'].iloc[-1]:.3f}")

In [ ]:
!find /content -name "results.csv"  # 搜索Colab中所有results.csv文件

In [ ]:
import pandas as pd

# 替换为你的日志文件路径
log_path = '/content/runs/detect/train2/results.csv'

# 读取日志数据
df = pd.read_csv(log_path)

# 打印关键指标（mAP@0.5是最直观的目标检测精度指标）
print(f"最后一轮训练集mAP@0.5: {df['metrics/mAP50(B)'].iloc[-1]:.3f}")  # 越接近1越好
#print(f"最后一轮验证集mAP@0.5: {df['val/metrics/mAP50(B)'].iloc[-1]:.3f}")

In [ ]:
# 选择一张验证集图像（可通过!ls /content/underwater_dataset/underwater_dataset/val/images查看）
test_image_path = '/content/underwater_dataset/underwater_dataset/val/images/img_99.jpg'

# 用训练好的模型预测
results = model(test_image_path)

# 保存带检测框的结果图
for r in results:
    r.save(filename='/content/test_result_85.jpg')  # 保存路径

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# 显示结果图
img = Image.open('/content/test_result_85.jpg')
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')  # 关闭坐标轴
plt.show()

In [ ]:
# 在验证集上评估模型
metrics = model.val(data='/content/underwater.yaml', device=0)

# 输出关键指标
print(f"mAP@0.5: {metrics.box.map50:.3f}")  # IoU=0.5时的平均精度（越高越好，>0.5即为可用）
print(f"精确率: {metrics.box.p:.3f}")       # 预测为目标的样本中，实际是目标的比例
print(f"召回率: {metrics.box.r:.3f}")       # 实际是目标的样本中，被预测到的比例

In [ ]:
# 打印每个类别的精确率和召回率
print("各类别精确率（P）：", metrics.box.p)  # 输出如 [0.715 0.339]（对应rock和pipe）
print("各类别召回率（R）：", metrics.box.r)  # 输出如 [0.196 0.429]

# 格式化显示（保留3位小数）
print(f"rock精确率: {metrics.box.p[0]:.3f}")  # 第一个类别（rock）的精确率
print(f"pipe精确率: {metrics.box.p[1]:.3f}")  # 第二个类别（pipe）的精确率
print(f"rock召回率: {metrics.box.r[0]:.3f}")
print(f"pipe召回率: {metrics.box.r[1]:.3f}")

In [ ]:
import numpy as np

# 计算平均值（所有类别的平均）
mean_precision = np.mean(metrics.box.p)  # 平均精确率
mean_recall = np.mean(metrics.box.r)     # 平均召回率

print(f"平均精确率: {mean_precision:.3f}")  # 如 (0.715+0.339)/2 ≈ 0.527
print(f"平均召回率: {mean_recall:.3f}")     # 如 (0.196+0.429)/2 ≈ 0.312

In [ ]:
# 复制最佳模型权重到云盘
!cp /content/underwater_runs/first_train/weights/best.pt /content/drive/MyDrive/

In [ ]:
model = YOLO('/content/drive/MyDrive/best.pt')

In [ ]:
# 覆盖原有的underwater.yaml，写入正确配置
with open('/content/underwater.yaml', 'w') as f:
    f.write('''
train: /content/underwater_dataset/train/images
val: /content/underwater_dataset/val/images
nc: 2
names: ['rock', 'pipe']
class_weights: [2.0, 1.0]
    ''')

# 验证配置文件内容是否正确
!cat /content/underwater.yaml

In [ ]:
model.train(
    data='/content/underwater.yaml',
    epochs=50,  # 增加轮次，让模型充分学习增强后的数据
    batch=8,
    imgsz=640,
    augment=True  # 关键：开启数据增强
)

In [ ]:
# 查看/content目录下是否有underwater_dataset文件夹
!ls /content

# 若存在，继续查看其内部结构（逐层确认）
!ls /content/underwater_dataset  # 查看一级子目录
!ls /content/underwater_dataset/underwater_dataset  # 查看二级子目录（如果有的话）

In [ ]:
# 根据实际路径修改下面的train和val路径
with open('/content/underwater.yaml', 'w') as f:
    f.write('''
train: /content/underwater_dataset/underwater_dataset/train/images  # 替换为你的实际训练集路径
val: /content/underwater_dataset/underwater_dataset/val/images      # 替换为你的实际验证集路径
nc: 2
names: ['rock', 'pipe']
class_weights: [2.0, 1.0]
    ''')

# 再次确认配置文件内容
!cat /content/underwater.yaml

In [ ]:
# 替换为配置文件中的val路径
val_images_path = '/content/underwater_dataset/underwater_dataset/val/images'

# 查看该路径下的图像数量（应≥1，否则需检查数据集）
!ls -l $val_images_path | wc -l  # 输出数量应>0，说明有图像

In [ ]:
model.train(
    data='/content/underwater.yaml',
    epochs=50,  # 增加轮次，让模型充分学习增强后的数据
    batch=8,
    imgsz=640,
    augment=True  # 关键：开启数据增强
)

In [ ]:
!find /content -name "results.csv"  # 搜索Colab中所有results.csv文件

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # 重新授权挂载，步骤和之前一样

In [ ]:
!pip install ultralytics  # 1分钟内完成，和第一次安装一样

In [ ]:
from ultralytics import YOLO
# 直接加载本地的best.pt，路径和你之前训练结果的路径一致
model = YOLO('/content/runs/detect/train3/weights/best.pt')

In [ ]:
# 查看Google Drive根目录下是否有模型文件（比如之前建议的yolov8_underwater_best.pt）
!ls /content/drive/MyDrive | grep ".pt"  # 搜索云盘里的.pt模型文件

In [ ]:
from ultralytics import YOLO
# 加载云盘里的模型（路径替换成你搜索到的实际文件名）
model = YOLO('/content/drive/MyDrive/best.pt')
print("模型加载成功！")

In [ ]:
# 用验证集的一张图测试（路径替换成你数据集的实际图片）
test_img_path = '/content/underwater_dataset/underwater_dataset/val/images/img_90.jpg'
results = model(test_img_path)
# 显示检测结果（能弹出图就说明成功）
results[0].show()

In [ ]:
# 先查看验证集images文件夹的完整路径（确保文件夹存在）
val_images_dir = '/content/underwater_dataset/underwater_dataset/val/images'
!ls -l $val_images_dir  # 列出文件夹里的所有文件（包括图片名）

In [ ]:
# 1. 先查看/content目录下所有文件，找到数据集文件夹（通常叫underwater_dataset）
print("=== /content目录下的文件 ===")
!ls /content

# 2. 假设找到文件夹叫"underwater_dataset"，查看其内部结构
dataset_root = '/content/underwater_dataset'
print(f"\n=== {dataset_root} 内部结构 ===")
!ls $dataset_root

# 3. 继续查看下一层，直到找到train/val文件夹
# （如果第二步输出里有train/val，就用dataset_root；如果没有，就继续进下一层）
# 比如第二步输出里有"underwater_dataset_v1"，就把dataset_root改成下面这样：
# dataset_root = '/content/underwater_dataset/underwater_dataset_v1'
# 然后查看其内部结构：
# !ls $dataset_root

In [ ]:
# 1. 替换成你云盘里压缩包的真实名称（比如underwater_dataset.zip）
zip_file_path = '/content/drive/MyDrive/underwater_dataset.zip'

# 2. 设定解压后的目标文件夹（会自动创建underwater_dataset文件夹）
extract_target = '/content/underwater_dataset'

# 3. 执行解压（-o 表示强制覆盖，避免之前的残留文件干扰）
!unzip -o $zip_file_path -d $extract_target

# 4. 解压后查看结果（确认是否生成了train/val文件夹）
print("\n=== 解压后的文件夹结构 ===")
!ls $extract_target  # 这里应该能看到train和val文件夹

In [ ]:
import os  # 导入os模块，用于文件操作

# 补充样本后，数据集的真实路径（根据你的实际情况修改）
train_dir = '/content/underwater_dataset/underwater_dataset/train/images'
val_dir = '/content/underwater_dataset/underwater_dataset/val/images'

# 验证路径是否有效（能看到图片数量说明路径正确）
print("训练集图片数量：", len(os.listdir(train_dir)))
print("验证集图片数量：", len(os.listdir(val_dir)))

In [ ]:
import os

# 写入最新配置（根据你的实际情况修改参数）
with open('/content/underwater.yaml', 'w') as f:
    f.write(f'''
train: {train_dir}  # 最新训练集路径
val: {val_dir}      # 最新验证集路径
nc: 2               # 类别数量（礁石+管道，保持不变）
names: ['rock', 'pipe']  # 类别名称（保持不变，确保与标注一致）
class_weights: [2.0, 1.0]  # 若想进一步提升礁石权重，可加大（如3.0）
    ''')

# 验证配置文件内容是否正确
print("\n=== 最新配置文件内容 ===")
!cat /content/underwater.yaml

In [ ]:
# 3. 执行调优训练（参数详解见注释）
results = model.train(
    data='/content/underwater.yaml',  # 已配置好的数据集文件
    epochs=80,                        # 延长轮次（比之前多30-50轮，确保学透）
    batch=8,                          # 保持批次大小（适配T4 GPU）
    imgsz=640,                        # 输入尺寸不变

    # 学习率调优：初始学习率减小，配合余弦衰减，避免破坏已有特征
    lr0=0.001,                        # 初始学习率（默认0.01，调小10倍，适合微调）
    cos_lr=True,                      # 余弦学习率衰减（后期学习率更小，优化更精细）

    # 数据增强：针对水下环境优化（抗光照、水色变化）
    augment=True,                     # 开启基础增强
    hsv_h=0.2,                        # 色调增强（0.015→0.2，适应不同水色）
    hsv_s=0.8,                        # 饱和度增强（0.7→0.8，模拟浑浊/清澈水域）
    hsv_v=0.5,                        # 亮度增强（0.4→0.5，适应深浅水区）
    degrees=15,                       # 旋转增强（0→15度，目标可能倾斜）
    translate=0.2,                    # 平移增强（0.1→0.2，目标位置更灵活）


    # 早停机制：验证集mAP连续20轮不提升则自动停止
    patience=20,

    # 保存设置：结果保存到云盘，避免丢失
    project='/content/drive/MyDrive/yolov8_tune_runs',  # 云盘路径
    name='tune_train',                                 # 本次训练文件夹名
    save_period=10                                     # 每10轮保存一次权重
)

In [ ]:
from ultralytics import YOLO
import os

# 加载之前的最佳模型（第31轮的best.pt，路径从结果中复制）
model = YOLO('/content/drive/MyDrive/yolov8_tune_runs/tune_train/weights/best.pt')

# 确认数据集路径（保持不变）
train_dir = '/content/underwater_dataset/underwater_dataset/train/images'
val_dir = '/content/underwater_dataset/underwater_dataset/val/images'

# 调整后的训练参数（核心：降低学习率、减弱增强、延长早停 patience）
results = model.train(
    data='/content/underwater.yaml',
    epochs=60,                # 减少轮次，避免过拟合
    batch=8,
    imgsz=640,
    lr0=0.0005,               # 学习率再降一半，更精细微调
    cos_lr=True,              # 保留余弦衰减，但初始值更低
    augment=True,
    # 减弱数据增强（更贴近真实水下场景）
    hsv_h=0.1,                # 色调增强从0.2→0.1
    hsv_s=0.5,                # 饱和度从0.8→0.5
    hsv_v=0.3,                # 亮度从0.5→0.3
    degrees=10,               # 旋转从15→10度
    translate=0.1,            # 平移从0.2→0.1
    patience=30,              # 早停耐心值从20→30，给模型更多时间适应
    project='/content/drive/MyDrive/yolov8_tune_runs',
    name='tune_train_v2',     # 新的训练文件夹，避免覆盖之前结果
    save_period=5             # 每5轮保存一次，方便回退到最优轮次
)

In [ ]:
from ultralytics import YOLO

# 加载当前最佳模型（tune_train_v2的best.pt）
model = YOLO('/content/drive/MyDrive/yolov8_tune_runs/tune_train_v2/weights/best.pt')

# 针对性提升精确率的训练参数
results = model.train(
    data='/content/underwater.yaml',
    epochs=50,                # 轮次适中，避免过拟合
    batch=8,
    imgsz=640,
    lr0=0.0003,               # 学习率再降，避免权重震荡导致误判
    cos_lr=True,
    augment=True,
    # 保持温和增强，不引入失真特征
    hsv_h=0.1,
    hsv_s=0.4,
    hsv_v=0.3,
    degrees=8,
    translate=0.1,
    # 关键：增加“框损失权重”，让模型更关注边界框准确性（减少误检）
    box=8.0,                  # 默认7.5→8.0，提升框损失权重
    cls=0.6,                  # 分类损失权重从0.5→0.6，让模型更关注类别判断
    patience=25,              # 早停耐心值适中
    project='/content/drive/MyDrive/yolov8_tune_runs',
    name='tune_train_v3',
    save_period=5
)

In [ ]:
# 备份上一次的best.pt（重命名为best_v3_backup.pt）
!cp /content/drive/MyDrive/yolov8_tune_runs/tune_train_v3/weights/best.pt /content/drive/MyDrive/yolov8_tune_runs/tune_train_v3/weights/best_v3_backup.pt
print("上一轮best.pt已备份为best_v3_backup.pt，可放心覆盖！")

In [ ]:
# 加载v3的最佳模型，微调召回率
model = YOLO('/content/drive/MyDrive/yolov8_tune_runs/tune_train_v3/weights/best.pt')

results = model.train(
    data='/content/underwater.yaml',
    epochs=40,                # 轮次减少，避免过拟合
    batch=8,
    imgsz=640,
    lr0=0.0002,               # 学习率再降，精细微调
    cos_lr=True,
    augment=True,
    # 温和增强，帮助模型识别模糊目标
    hsv_h=0.1,
    hsv_s=0.4,
    hsv_v=0.3,
    degrees=8,
    translate=0.1,
    # 轻微降低框损失权重，平衡召回率
    box=7.7,                  # 从8.0回调到7.7（比默认7.5略高，仍保精确率）
    cls=0.55,                 # 分类损失从0.6回调到0.55
    patience=20,              # 早停耐心值适中
    project='/content/drive/MyDrive/yolov8_tune_runs',
    name='tune_train_v4_balance',
    save_period=5
)

# 评估平衡后的指标
balance_metrics = model.val()
print(f"平衡后礁石召回率: {balance_metrics.box.r[0]:.3f}")  # 目标回升到0.28以上
print(f"平衡后平均精确率: {balance_metrics.box.p.mean():.3f}")  # 目标保持在0.65以上

In [ ]:
# 源路径：v3的最佳模型
source_path = '/content/drive/MyDrive/yolov8_tune_runs/tune_train_v3/weights/best.pt'
# 目标路径：最终保存的best.pt（云盘根目录，永久保存）
target_path = '/content/drive/MyDrive/best.pt'

# 执行覆盖（-f 强制覆盖，避免提示确认）
!cp -f $source_path $target_path

print(f"✅ 最终模型已覆盖保存为best.pt！")
print(f"路径：{target_path}")

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

# 加载覆盖后的最终模型
final_model = YOLO('/content/drive/MyDrive/best.pt')

# 用验证集图片测试
test_img = '/content/underwater_dataset/underwater_dataset/val/images/img_89.jpg'  # 替换为你的真实图片
results = final_model(test_img)

# 打印模型信息和检测结果
print(f"模型加载成功！检测到目标数量：{len(results[0].boxes)}")
# 显示检测框
plt.figure(figsize=(10, 8))
plt.imshow(results[0].plot())
plt.axis('off')
plt.show()

In [ ]:
from ultralytics import YOLO

# 加载最终保存的best.pt
model = YOLO('/content/drive/MyDrive/best.pt')

# 1. 查看模型配置信息（类别、输入尺寸等）
print("=== 模型基础配置 ===")
print(f"模型类别数（nc）：{model.model.nc}")  # 应输出2（rock+pipe）
print(f"类别名称（names）：{model.model.names}")  # 应输出['rock', 'pipe']
print(f"默认输入尺寸（imgsz）：{model.args['imgsz']}")  # 应输出640
print(f"模型层数/参数：{model.info()['layers']}层 / {model.info()['parameters']:.2f}M参数")  # 应约72层/3.01M参数

In [ ]:
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt

# 加载最终的best.pt
model = YOLO('/content/drive/MyDrive/best.pt')

# ---------------------- 第一步：确认模型基础配置（类别、输入尺寸） ----------------------
print("=== 1. 模型基础配置（确认是否为水下检测模型） ===")
# 1. 类别数和类别名称（核心，确认不是默认COCO模型）
print(f"类别数（nc）：{model.model.nc}")  # 正确应输出 2（rock+pipe）
print(f"类别名称：{model.model.names}")    # 正确应输出 {0: 'rock', 1: 'pipe'}
# 2. 默认输入尺寸（确认是640）
print(f"默认输入尺寸（imgsz）：{model.args['imgsz']}")  # 正确应输出 640
# 3. 模型结构（层数、参数，用info()正确取值）
model_info = model.info()  # 返回元组：(层数, 参数数, 梯度数, GFLOPs)
print(f"模型层数：{model_info[0]} 层")          # 正确应约129层（不同YOLOv8版本略有差异）
print(f"模型参数：{model_info[1]/1e6:.2f} M")   # 正确应约3.01 M（与训练时一致）
print(f"计算量（GFLOPs）：{model_info[3]:.1f}")  # 正确应约8.2 GFLOPs


# ---------------------- 第二步：验证性能指标（确认是否为v3的最优模型） ----------------------
print("\n=== 2. 验证集性能指标（确认是否匹配v3的效果） ===")
# 在验证集上跑指标，对比v3的核心结果（精确率0.739、mAP@0.5=0.356）
metrics = model.val(
    data='/content/underwater.yaml',  # 用你配置的数据集
    imgsz=640,
    batch=8
)
# 输出关键指标（与v3对比，波动±0.03内均正常）
print(f"平均精确率（P）：{metrics.box.p.mean():.3f}")  # 正确应≈0.739
print(f"礁石召回率（R）：{metrics.box.r[0]:.3f}")        # 正确应≈0.232
print(f"管道召回率（R）：{metrics.box.r[1]:.3f}")        # 正确应≈0.429
print(f"验证集mAP@0.5：{metrics.box.map50:.3f}")        # 正确应≈0.356


# ---------------------- 第三步：可视化检测（直观确认功能正常） ----------------------
print("\n=== 3. 可视化检测结果（确认无错检漏检异常） ===")
# 选2张验证集图片测试（替换为你的val/images路径）
val_img_dir = '/content/underwater_dataset/underwater_dataset/val/images'
val_imgs = [os.path.join(val_img_dir, img) for img in os.listdir(val_img_dir)[:2]]

for img_path in val_imgs:
    results = model(img_path)
    # 显示带检测框的图片
    plt.figure(figsize=(10, 8))
    plt.imshow(results[0].plot())  # plot()自动画框+类别+置信度
    plt.title(f"检测结果：{os.path.basename(img_path)}")
    plt.axis('off')
    plt.show()

In [ ]:
# 查看项目目录下的所有文件（替换为你的仓库名）
REPO_NAME = 'yolov8-uw-rp-detection'
!ls -l /content/{REPO_NAME}/weights/      # 检查是否有 best.pt
!ls -l /content/{REPO_NAME}/config/       # 检查是否有 underwater.yaml
!ls -l /content/{REPO_NAME}/train_code/   # 检查是否有 ipynb 文件
!ls -l /content/{REPO_NAME}/test_data/    # 检查是否有测试图片

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # 执行后按提示授权，确保输出 "Mounted at /content/drive"

In [ ]:
# 1. 搜索 best.pt（包括云盘）
print("=== 查找 best.pt（含云盘） ===")
!find /content/drive -name "best.pt"  # 搜索云盘所有目录

# 2. 搜索 underwater.yaml（包括云盘）
print("\n=== 查找 underwater.yaml（含云盘） ===")
!find /content/drive -name "underwater.yaml"

# 3. 确认训练笔记本：直接看当前打开的笔记本名称
print("\n=== 当前打开的 Colab 笔记本名称 ===")
print("请查看 Colab 页面顶部的文件名（如 'Untitled0.ipynb'）")

# 4. 搜索验证集图片（包括云盘）
print("\n=== 查找验证集图片（含云盘） ===")
!find /content/drive -path "*/val/images/*.jpg" | head -1

In [ ]:
REPO_NAME = 'yolov8-uw-rp-detection'
project_root = f'/content/{REPO_NAME}'

# 云盘笔记本的真实路径（默认都在 Colab Notebooks 文件夹）
notebook_path = '/content/drive/MyDrive/Colab Notebooks/YOLOv8_underwater_detection.ipynb'

# 复制到 train_code 文件夹
!cp "{notebook_path}" {project_root}/train_code/

# 检查是否成功
print("=== 检查 train_code 文件夹 ===")
!ls {project_root}/train_code/  # 能看到 ipynb 文件就对了

In [ ]:
os.chdir(project_root)

# 撤销上一次包含令牌的提交（关键！）
!git reset --soft HEAD~1  # 回到上一次提交前的状态

# 重新添加所有文件（此时笔记本已无令牌）
!git add .

# 重新提交
!git commit -m "complete project: remove secrets, add all files"

# 再次推送
!git push -u origin master -f

[master 9753ce7] complete project: remove secrets, add all files
 2 files changed, 1 insertion(+)
 create mode 100644 train_code/YOLOv8_underwater_detection.ipynb
 create mode 100644 weights/best.pt
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 8.29 MiB | 6.83 MiB/s, done.
Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
